In [1]:
import os
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn import linear_model
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


In [2]:
class SubNet(nn.Module):
    '''
    The subnetwork that is used in TFN for video and audio in the pre-fusion stage
    '''

    def __init__(self, in_size, hidden_size, dropout):
        '''
        Args:
            in_size: input dimension
            hidden_size: hidden layer dimension
            dropout: dropout probability
        Output:
            (return value in forward) a tensor of shape (batch_size, hidden_size)
        '''
        super(SubNet, self).__init__()
        self.norm = nn.BatchNorm1d(in_size)
        self.drop = nn.Dropout(p=dropout)
        self.linear_1 = nn.Linear(in_size, hidden_size)
        self.linear_2 = nn.Linear(hidden_size, hidden_size)
        self.linear_3 = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        '''
        Args:
            x: tensor of shape (batch_size, in_size)
        '''
        normed = self.norm(x)
        dropped = self.drop(normed)
        y_1 = F.relu(self.linear_1(dropped))
        y_2 = F.relu(self.linear_2(y_1))
        y_3 = F.relu(self.linear_3(y_2))

        return y_2


class TFN(nn.Module):
    '''
    Implements the Tensor Fusion Networks for multimodal sentiment analysis as is described in:
    Zadeh, Amir, et al. "Tensor fusion network for multimodal sentiment analysis." EMNLP 2017 Oral.
    '''

    def __init__(self, input_dims, hidden_dims, dropouts, post_fusion_dim):
        '''
        Args:
            input_dims - a length-3 tuple, contains (audio_dim, video_dim, text_dim)
            hidden_dims - another length-3 tuple, similar to input_dims
            text_out - int, specifying the resulting dimensions of the text subnetwork
            dropouts - a length-4 tuple, contains (audio_dropout, video_dropout, text_dropout, post_fusion_dropout)
            post_fusion_dim - int, specifying the size of the sub-networks after tensorfusion
        Output:
            (return value in forward) a scalar value between -3 and 3
        '''
        super(TFN, self).__init__()

        # dimensions are specified in the order of audio, video and text
        self.audio_in = input_dims[0]
        self.video_in = input_dims[1]
        self.pert_in = input_dims[2]
        self.iris_in = input_dims[3]
        self.physical_in = input_dims[4]
        self.macbert_in = input_dims[5]
        self.robert_in = input_dims[6]

        self.audio_hidden = hidden_dims[0]
        self.video_hidden = hidden_dims[1]
        self.pert_hidden = hidden_dims[2]
        self.iris_hidden = hidden_dims[3]
        self.physical_hidden = hidden_dims[4]
        self.macbert_hidden = hidden_dims[5]
        self.robert_hidden = hidden_dims[6]

        self.post_fusion_dim = post_fusion_dim

        self.audio_prob = dropouts[0]
        self.video_prob = dropouts[1]
        self.pert_prob = dropouts[2]
        self.iris_prob = dropouts[3]
        self.physical_prob = dropouts[4]
        self.macbert_prob = dropouts[5]
        self.robert_prob = dropouts[6]
        self.post_fusion_prob = dropouts[7]
        
        # define the pre-fusion subnetworks
        # self.audio_subnet = SubNet(self.audio_in, self.audio_hidden, self.audio_prob)
        # self.video_subnet = SubNet(self.video_in, self.video_hidden, self.video_prob)
        # self.text_subnet = TextSubNet(self.text_in, self.text_hidden, self.text_out, dropout=self.text_prob)
        # define the pre-fusion subnetworks
        self.audio_subnet = SubNet(self.audio_in, self.audio_hidden, self.audio_prob)
        self.video_subnet = SubNet(self.video_in, self.video_hidden, self.video_prob)
        self.pert_subnet = SubNet(self.pert_in, self.pert_hidden, self.pert_prob)
        self.iris_subnet = SubNet(self.iris_in, self.iris_hidden, self.iris_prob)
        self.physical_subnet = SubNet(self.physical_in, self.physical_hidden, self.physical_prob)
        self.macbert_subnet = SubNet(self.macbert_in, self.macbert_hidden, self.macbert_prob)
        self.robert_subnet = SubNet(self.robert_in, self.robert_hidden, self.robert_prob)

        # define the post_fusion layers
        self.post_fusion_dropout = nn.Dropout(p=self.post_fusion_prob)
        self.post_fusion_layer_1 = nn.Linear((self.macbert_hidden + 1) * (self.robert_hidden + 1) *\
        (self.pert_hidden + 1) * (self.video_hidden + 1) * (self.audio_hidden + 1)* \
        (self.iris_hidden + 1)* (self.physical_hidden + 1), self.post_fusion_dim)
        self.post_fusion_layer_2 = nn.Linear(self.post_fusion_dim, self.post_fusion_dim)
        self.post_fusion_layer_3 = nn.Linear(self.post_fusion_dim, 1)

        # in TFN we are doing a regression with constrained output range: (-3, 3), hence we'll apply sigmoid to output
        # shrink it to (0, 1), and scale\shift it back to range (-3, 3)
        # self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False)
        # self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)

    def forward(self, audio_x, video_x, pert_x, iris_x, physical_x, macbert_x, robert_x):
        '''
        Args:
            audio_x: tensor of shape (batch_size, audio_in)
            video_x: tensor of shape (batch_size, video_in)
            text_x: tensor of shape (batch_size, sequence_len, text_in)
        '''
        audio_h = self.audio_subnet(audio_x)
        video_h = self.video_subnet(video_x)
        pert_h = self.pert_subnet(pert_x)
        iris_h = self.iris_subnet(iris_x)
        physical_h = self.physical_subnet(physical_x)
        macbert_h = self.macbert_subnet(macbert_x)
        robert_h = self.robert_subnet(robert_x)
        batch_size = audio_h.data.shape[0]

        # next we perform "tensor fusion", which is essentially appending 1s to the tensors and take Kronecker product
        if audio_h.is_cuda:
            DTYPE = torch.cuda.FloatTensor
        else:
            DTYPE = torch.FloatTensor

        _audio_h = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), audio_h), dim=1)
        _video_h = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), video_h), dim=1)
        _pert_h = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), pert_h), dim=1)
        _iris_h = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), iris_h), dim=1)
        _physical_h = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), physical_h), dim=1)
        _macbert_h = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), macbert_h), dim=1)
        _robert_h = torch.cat((Variable(torch.ones(batch_size, 1).type(DTYPE), requires_grad=False), robert_h), dim=1)
        
        # _audio_h has shape (batch_size, audio_in + 1), _video_h has shape (batch_size, _video_in + 1)
        # we want to perform outer product between the two batch, hence we unsqueenze them to get
        # (batch_size, audio_in + 1, 1) X (batch_size, 1, video_in + 1)
        # fusion_tensor will have shape (batch_size, audio_in + 1, video_in + 1)
        fusion_tensor = torch.bmm(_audio_h.unsqueeze(2), _video_h.unsqueeze(1))

        # next we do kronecker product between fusion_tensor and _text_h. This is even trickier
        # we have to reshape the fusion tensor during the computation
        # in the end we don't keep the 3-D tensor, instead we flatten it
        fusion_tensor = fusion_tensor.view(-1, (self.audio_hidden + 1) * (self.video_hidden + 1), 1)
        fusion_tensor = torch.bmm(fusion_tensor, _pert_h.unsqueeze(1))
        fusion_tensor = fusion_tensor.view(-1, (self.audio_hidden + 1) * (self.video_hidden + 1) * (self.pert_hidden + 1), 1)
        fusion_tensor = torch.bmm(fusion_tensor, _iris_h.unsqueeze(1))
        fusion_tensor = fusion_tensor.view(-1, (self.audio_hidden + 1) * (self.video_hidden + 1) * \
            (self.pert_hidden + 1) * (self.iris_hidden + 1), 1)
        fusion_tensor = torch.bmm(fusion_tensor, _macbert_h.unsqueeze(1))
        fusion_tensor = fusion_tensor.view(-1, (self.audio_hidden + 1) * (self.video_hidden + 1) * \
            (self.pert_hidden + 1) * (self.iris_hidden + 1) * (self.macbert_hidden + 1), 1)
        fusion_tensor = torch.bmm(fusion_tensor, _robert_h.unsqueeze(1))
        fusion_tensor = fusion_tensor.view(-1, (self.audio_hidden + 1) * (self.video_hidden + 1) * \
            (self.pert_hidden + 1) * (self.iris_hidden + 1) * (self.macbert_hidden + 1) * (self.robert_hidden + 1), 1)
        
        fusion_tensor = torch.bmm(fusion_tensor, _physical_h.unsqueeze(1)).view(batch_size, -1)

        post_fusion_dropped = self.post_fusion_dropout(fusion_tensor)
        post_fusion_y_1 = F.relu(self.post_fusion_layer_1(post_fusion_dropped))
        post_fusion_y_2 = F.relu(self.post_fusion_layer_2(post_fusion_y_1))
        post_fusion_y_3 = torch.sigmoid(self.post_fusion_layer_3(post_fusion_y_2))
        output = post_fusion_y_3  # * self.output_range + self.output_shift

        return output

In [3]:
def get_face_landmarks(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3])
    # print(df.head(10))
    df_x = df['x']
    df_y = df['y']
    df_z = df['z']
    face_x = df_x.values.tolist()
    face_y = df_y.values.tolist()
    face_z = df_z.values.tolist()

    print("There are {num} faces recorded from this sample.".format(num = np.array(face_x).shape[0] / 468))
    print(np.array(face_x).shape)

    return face_x, face_y, face_z, (np.array(face_x).shape[0] / 468)

def get_avg_face_landmarks(x, y, z, face_num):
    avg_x = []
    avg_y = []
    avg_z = []

    for i in range(468):
        temp = x[i:len(x):468]
        avg_x.append(np.sum(temp)/face_num)
    for i in range(468):
        temp = y[i:len(x):468]
        avg_y.append(np.sum(temp)/face_num)
    for i in range(468):
        temp = z[i:len(x):468]
        avg_z.append(np.sum(temp)/face_num)

    print(np.array(avg_x).shape)
    return avg_x, avg_y, avg_z

def get_iris_landmarks(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3,4])
    # print(df.head(10))
    df_leftx = df['left_x']
    df_lefty = df['left_y']
    df_rightx = df['right_x']
    df_righty = df['right_y']
    iris_left_x = df_leftx.values.tolist()
    iris_left_y = df_lefty.values.tolist()
    iris_right_x = df_rightx.values.tolist()
    iris_right_y = df_righty.values.tolist()

    print("There are {num} irises recorded from this sample.".format(num = np.array(df_leftx).shape[0]))
    print(np.array(iris_left_x).shape)

    return iris_left_x, iris_left_y, iris_right_x, iris_right_y, (np.array(df_leftx).shape[0])

def get_avg_iris_landmarks(leftx, lefty, rightx, righty, iris_num):
    avg_leftx = np.sum(leftx) / iris_num
    avg_lefty = np.sum(lefty) / iris_num
    avg_rightx = np.sum(rightx) / iris_num
    avg_righty = np.sum(righty) / iris_num

    return avg_leftx, avg_lefty, avg_rightx, avg_righty

def get_physical_index(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_index = df['TSfresh'].values.tolist()

    return df_index

def get_voice_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_mfcc = df['MFCC'].values.tolist()
    
    return df_mfcc

def get_text_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3])
    df_pert = df['pert'].values.tolist()
    df_roberta = df['roberta'].values.tolist()
    df_macberta = df['macberta'].values.tolist()
    return df_pert, df_roberta, df_macberta

def get_house_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_house = df['house'].values.tolist()
    
    return df_house

if __name__ == '__main__':
    face = []
    iris = []
    physical_index = []
    mfcc_ft = []
    pert = []
    roberta = []
    macberta = []
    house = []


    labels = []
    # 标签
    label = {b'healthy': 0, b'unhealthy': 1}

    root = os.path.dirname(os.getcwd())
    print("root direction", root)

    dataset_class = os.listdir(root + '/Dataset')
    print(dataset_class)

    # 遍历数据集的人脸信息
    for sample_class in dataset_class:
        sample_class_path = root + '/Dataset' + '/' + sample_class
        # print(sample_class_path)
        sample_file = os.listdir(sample_class_path)
        # print(sample_file)

        for detail in sample_file:
            detail_path = sample_class_path + '/' + detail
            sample_detail = os.listdir(detail_path)
            print(detail_path)

            for csv_file in sample_detail:
                if 'face_landmarks' in csv_file:
                    # 打开csv文件并读取人脸信息
                    print(detail_path + '/' + csv_file)
                    face_x, face_y, face_z, face_num = get_face_landmarks(detail_path + '/' + csv_file)
                    avg_x, avg_y, avg_z = get_avg_face_landmarks(face_x, face_y, face_z, face_num)
                    temp_face = avg_x + avg_y + avg_z
                    face.append(temp_face)
                    print(np.array(face).shape)
                    # 加上标签
                    if sample_class == 'healthy':
                        labels.append(0)
                    elif sample_class == 'unhealthy':
                        labels.append(1)

                if 'iris_location' in csv_file:
                    print(detail_path + '/' + csv_file)
                    left_x, left_y, right_x, right_y, iris_num = get_iris_landmarks(detail_path + '/' + csv_file)
                    
                    # avg_leftx, avg_lefty, avg_rightx, avg_righty = get_avg_iris_landmarks(left_x, left_y, right_x, right_y, iris_num)
                    # temp_iris = avg_leftx + avg_lefty + avg_rightx + avg_righty
                    # print(np.array(temp_iris).shape)
                    iris.append(left_x)
                    iris.append(left_y)
                    iris.append(right_x)
                    iris.append(right_y)
                    
                    print(np.array(iris).shape)

                if 'Physical_feature' in csv_file:
                    print(detail_path + '/' + csv_file)
                    temp_index = get_physical_index(detail_path + '/' + csv_file)
                    physical_index.append(temp_index)
                    print(np.array(physical_index).shape)

                if 'Voice_feature' in csv_file:
                    print(detail_path + '/' + csv_file)
                    temp_mfcc = get_voice_feature(detail_path + '/' + csv_file)
                    mfcc_ft.append(temp_mfcc)
                    print(np.array(mfcc_ft).shape)

                if 'text_feature' in csv_file:
                    print(detail_path + '/' + csv_file)
                    temp_pert, temp_roberta, temp_macberta = get_text_feature(detail_path + '/' + csv_file)
                    pert.append(temp_pert)
                    roberta.append(temp_roberta)
                    macberta.append(temp_macberta)
                    print(np.array(pert).shape, np.array(roberta).shape, np.array(macberta).shape)


                if 'house' in csv_file:
                    print(detail_path + '/' + csv_file)
                    temp_house = get_house_feature(detail_path + '/' + csv_file)
                    house.append(temp_house)
                    print(np.array(house).shape)

    iris = np.array(iris).reshape(-1, 684*4)            
    # iris = np.array(iris).reshape(-1, 1)
    print("Face set:{face}".format(face = np.array(face).shape))
    print("Iris set:{iris}".format(iris = np.array(iris).shape))
    print("Physical index set:{index}".format(index = np.array(physical_index).shape))
    print("MFCC set:{mfcc}".format(mfcc = np.array(mfcc_ft).shape))
    print("Pert set:{pert}, Roberta:{robert} and Macberta:{mac}".format(pert = np.array(pert).shape, \
        robert = np.array(roberta).shape, mac = np.array(macberta).shape))
    print("House set:{house}".format(house = np.array(house).shape))
    print("Label set:{label}".format(label = np.array(labels).shape))

root direction f:\Project\AMH
['healthy', 'unhealthy']
f:\Project\AMH/Dataset/healthy/00002-0101
f:\Project\AMH/Dataset/healthy/00002-0101/face_landmarks.csv
There are 209.0 faces recorded from this sample.
(97812,)
(468,)
(1, 1404)
f:\Project\AMH/Dataset/healthy/00002-0101/house.csv
(1, 1760)
f:\Project\AMH/Dataset/healthy/00002-0101/iris_location.csv
There are 684 irises recorded from this sample.
(684,)
(4, 684)
f:\Project\AMH/Dataset/healthy/00002-0101/Physical_feature.csv
(1, 4722)
f:\Project\AMH/Dataset/healthy/00002-0101/text_feature.csv
(1, 1024) (1, 1024) (1, 1024)
f:\Project\AMH/Dataset/healthy/00002-0101/Voice_feature.csv
(1, 9828)
f:\Project\AMH/Dataset/healthy/00002-0102
f:\Project\AMH/Dataset/healthy/00002-0102/face_landmarks.csv
There are 224.0 faces recorded from this sample.
(104832,)
(468,)
(2, 1404)
f:\Project\AMH/Dataset/healthy/00002-0102/house.csv
(2, 1760)
f:\Project\AMH/Dataset/healthy/00002-0102/iris_location.csv
There are 684 irises recorded from this sample.


In [3]:
def get_face_landmarks(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3])
    # print(df.head(10))
    df_x = df['x']
    df_y = df['y']
    df_z = df['z']
    face_x = df_x.values.tolist()
    face_y = df_y.values.tolist()
    face_z = df_z.values.tolist()

    print("There are {num} faces recorded from this sample.".format(num = np.array(face_x).shape[0] / 468))
    print(np.array(face_x).shape)

    return face_x, face_y, face_z, (np.array(face_x).shape[0] / 468)

def get_avg_face_landmarks(x, y, z, face_num):
    avg_x = []
    avg_y = []
    avg_z = []

    for i in range(468):
        temp = x[i:len(x):468]
        avg_x.append(np.sum(temp)/face_num)
    for i in range(468):
        temp = y[i:len(x):468]
        avg_y.append(np.sum(temp)/face_num)
    for i in range(468):
        temp = z[i:len(x):468]
        avg_z.append(np.sum(temp)/face_num)

    print(np.array(avg_x).shape)
    return avg_x, avg_y, avg_z

def get_iris_landmarks(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3,4])
    # print(df.head(10))
    df_leftx = df['left_x']
    df_lefty = df['left_y']
    df_rightx = df['right_x']
    df_righty = df['right_y']
    iris_left_x = df_leftx.values.tolist()
    iris_left_y = df_lefty.values.tolist()
    iris_right_x = df_rightx.values.tolist()
    iris_right_y = df_righty.values.tolist()

    print("There are {num} irises recorded from this sample.".format(num = np.array(df_leftx).shape[0]))
    print(np.array(iris_left_x).shape)

    return iris_left_x, iris_left_y, iris_right_x, iris_right_y, (np.array(df_leftx).shape[0])

def get_avg_iris_landmarks(leftx, lefty, rightx, righty, iris_num):
    avg_leftx = np.sum(leftx) / iris_num
    avg_lefty = np.sum(lefty) / iris_num
    avg_rightx = np.sum(rightx) / iris_num
    avg_righty = np.sum(righty) / iris_num

    return avg_leftx, avg_lefty, avg_rightx, avg_righty

def get_physical_index(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_index = df['TSfresh'].values.tolist()

    return df_index

def get_voice_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[1])
    df_mfcc = df['MFCC'].values.tolist()
    
    return df_mfcc

def get_text_feature(csv_path):
    df = pd.read_csv(csv_path, usecols=[2])
    df_text = df['roberta'].values.tolist()
    return df_text

if __name__ == '__main__':
    face = []
    iris = []
    physical_index = []
    mfcc_ft = []
    text = []

    labels = []
    # 标签
    label = {b'healthy': 0, b'unhealthy': 1}

    root = os.path.dirname(os.getcwd())
    print("root direction", root)

    dataset_class = os.listdir(root + '/Dataset')
    print(dataset_class)

    # 遍历数据集的人脸信息
    for sample_class in dataset_class:
        sample_class_path = root + '/Dataset' + '/' + sample_class
        # print(sample_class_path)
        sample_file = os.listdir(sample_class_path)
        # print(sample_file)

        for detail in sample_file:
            detail_path = sample_class_path + '/' + detail
            sample_detail = os.listdir(detail_path)
            print(detail_path)

            for csv_file in sample_detail:
                if 'face_landmarks' in csv_file:
                    # 打开csv文件并读取人脸信息
                    print(detail_path + '/' + csv_file)
                    face_x, face_y, face_z, face_num = get_face_landmarks(detail_path + '/' + csv_file)
                    avg_x, avg_y, avg_z = get_avg_face_landmarks(face_x, face_y, face_z, face_num)
                    temp_face = avg_x + avg_y + avg_z
                    face.append(temp_face)
                    
                    # 加上标签
                    if sample_class == 'healthy':
                        labels.append(0)
                    elif sample_class == 'unhealthy':
                        labels.append(1)

                if 'iris_location' in csv_file:
                    print(detail_path + '/' + csv_file)
                    left_x, left_y, right_x, right_y, iris_num = get_iris_landmarks(detail_path + '/' + csv_file)
                    
                    avg_leftx, avg_lefty, avg_rightx, avg_righty = get_avg_iris_landmarks(left_x, left_y, right_x, right_y, iris_num)
                    # temp_iris = avg_leftx + avg_lefty + avg_rightx + avg_righty
                    # print(np.array(temp_iris).shape)
                    iris.append(avg_leftx)
                    iris.append(avg_lefty)
                    iris.append(avg_rightx)
                    iris.append(avg_righty)
                    print(np.array(iris).shape)

                if 'Physical_feature' in csv_file:
                    print(detail_path + '/' + csv_file)
                    temp_index = get_physical_index(detail_path + '/' + csv_file)
                    physical_index.append(temp_index)
                    print(np.array(physical_index).shape)

                if 'Voice_feature' in csv_file:
                    print(detail_path + '/' + csv_file)
                    temp_mfcc = get_voice_feature(detail_path + '/' + csv_file)
                    mfcc_ft.append(temp_mfcc)
                    print(np.array(mfcc_ft).shape)

                if 'text_feature' in csv_file:
                    print(detail_path + '/' + csv_file)
                    temp_text = get_text_feature(detail_path + '/' + csv_file)
                    text.append(temp_text)
                    print(np.array(text).shape)


                
    iris = np.array(iris).reshape(-1, 4)
    print("Face set:{face}".format(face = np.array(face).shape))
    print("Iris set:{iris}".format(iris = np.array(iris).shape))
    print("Physical index set:{index}".format(index = np.array(physical_index).shape))
    print("MFCC set:{mfcc}".format(mfcc = np.array(mfcc_ft).shape))
    print("Text set:{text}".format(text = np.array(text).shape))
    print("Label set:{label}".format(label = np.array(labels).shape))

root direction f:\Project\AMH
['healthy', 'unhealthy']
f:\Project\AMH/Dataset/healthy/00002-0101
f:\Project\AMH/Dataset/healthy/00002-0101/face_landmarks.csv
There are 209.0 faces recorded from this sample.
(97812,)
(468,)
f:\Project\AMH/Dataset/healthy/00002-0101/iris_location.csv
There are 209 irises recorded from this sample.
(209,)
(4,)
f:\Project\AMH/Dataset/healthy/00002-0101/Physical_feature.csv
(1, 4722)
f:\Project\AMH/Dataset/healthy/00002-0101/text_feature.csv
(1, 1024)
f:\Project\AMH/Dataset/healthy/00002-0101/Voice_feature.csv
(1, 9828)
f:\Project\AMH/Dataset/healthy/00002-0102
f:\Project\AMH/Dataset/healthy/00002-0102/face_landmarks.csv
There are 224.0 faces recorded from this sample.
(104832,)
(468,)
f:\Project\AMH/Dataset/healthy/00002-0102/iris_location.csv
There are 224 irises recorded from this sample.
(224,)
(8,)
f:\Project\AMH/Dataset/healthy/00002-0102/Physical_feature.csv
(2, 4722)
f:\Project\AMH/Dataset/healthy/00002-0102/text_feature.csv
(2, 1024)
f:\Project\AMH

In [4]:
face_x_train, face_x_test, face_y_train, face_y_test = \
        train_test_split(face, labels, random_state=0, train_size=0.7)
iris_x_train, iris_x_test, iris_y_train, iris_y_test = \
        train_test_split(iris, labels, random_state=0, train_size=0.7)
physical_index_x_train, physical_index_x_test, physical_index_y_train, physical_index_y_test = \
        train_test_split(physical_index, labels, random_state=0, train_size=0.7)
mfcc_ft_x_train, mfcc_ft_x_test, mfcc_ft_y_train, mfcc_ft_y_test = \
        train_test_split(mfcc_ft, labels, random_state=0, train_size=0.7)
pert_x_train, pert_x_test, pert_y_train, pert_y_test = \
        train_test_split(pert, labels, random_state=0, train_size=0.7)
roberta_x_train, roberta_x_test, roberta_y_train, roberta_y_test = \
        train_test_split(roberta, labels, random_state=0, train_size=0.7)
macberta_x_train, macberta_x_test, macberta_y_train, macberta_y_test = \
        train_test_split(macberta, labels, random_state=0, train_size=0.7)

ros = RandomOverSampler(random_state=0)
sm = SMOTE(random_state=0)
face_x_train, face_y_train = sm.fit_resample(face_x_train, face_y_train)
iris_x_train, iris_y_train = sm.fit_resample(iris_x_train, iris_y_train)
physical_index_x_train, physical_index_y_train = sm.fit_resample(physical_index_x_train, \
        physical_index_y_train)
mfcc_ft_x_train, mfcc_ft_y_train = sm.fit_resample(mfcc_ft_x_train, mfcc_ft_y_train)
pert_x_train, pert_y_train = sm.fit_resample(pert_x_train, pert_y_train)
roberta_x_train, roberta_y_train = sm.fit_resample(roberta_x_train, roberta_y_train)
macberta_x_train, macberta_y_train = sm.fit_resample(macberta_x_train, macberta_y_train)

dtype = torch.FloatTensor

face_x_train, face_x_test, face_y_train, face_y_test = np.array(face_x_train), np.array(face_x_test),\
        np.array(face_y_train), np.array(face_y_test)
iris_x_train, iris_x_test, iris_y_train, iris_y_test = np.array(iris_x_train), np.array(iris_x_test),\
        np.array(iris_y_train), np.array(iris_y_test)
physical_index_x_train, physical_index_x_test, physical_index_y_train, physical_index_y_test = \
        np.array(physical_index_x_train), np.array(physical_index_x_test), \
            np.array(physical_index_y_train), np.array(physical_index_y_test)
mfcc_ft_x_train, mfcc_ft_x_test, mfcc_ft_y_train, mfcc_ft_y_test = np.array(mfcc_ft_x_train), \
        np.array(mfcc_ft_x_test), np.array(mfcc_ft_y_train), np.array(mfcc_ft_y_test)
pert_x_train, pert_x_test, pert_y_train, pert_y_test = np.array(pert_x_train), np.array(pert_x_test),\
        np.array(pert_y_train), np.array(pert_y_test)
roberta_x_train, roberta_x_test, roberta_y_train, roberta_y_test = np.array(roberta_x_train), np.array(roberta_x_test),\
        np.array(roberta_y_train), np.array(roberta_y_test)
macberta_x_train, macberta_x_test, macberta_y_train, macberta_y_test = np.array(macberta_x_train), np.array(macberta_x_test),\
        np.array(macberta_y_train), np.array(macberta_y_test)

pert_x_train, pert_y_train = torch.from_numpy(pert_x_train).type(dtype), \
        torch.from_numpy(pert_y_train).type(dtype)
roberta_x_train, roberta_y_train = torch.from_numpy(roberta_x_train).type(dtype), \
        torch.from_numpy(roberta_y_train).type(dtype)
macberta_x_train, macberta_y_train = torch.from_numpy(macberta_x_train).type(dtype), \
        torch.from_numpy(macberta_y_train).type(dtype)
mfcc_ft_x_train, mfcc_ft_y_train = torch.from_numpy(mfcc_ft_x_train).type(dtype), \
        torch.from_numpy(mfcc_ft_y_train).type(dtype)
face_x_train, face_y_train = torch.from_numpy(face_x_train).type(dtype), \
        torch.from_numpy(face_y_train).type(dtype)
iris_x_train, iris_y_train = torch.from_numpy(iris_x_train).type(dtype), \
        torch.from_numpy(iris_y_train).type(dtype)
physical_index_x_train, physical_index_y_train = torch.from_numpy(physical_index_x_train).type(dtype), \
        torch.from_numpy(physical_index_y_train).type(dtype)

pert_x_test, pert_y_test = torch.from_numpy(pert_x_test).type(dtype), \
        torch.from_numpy(pert_y_test).type(dtype)
roberta_x_test, roberta_y_test = torch.from_numpy(roberta_x_test).type(dtype), \
        torch.from_numpy(roberta_y_test).type(dtype)
macberta_x_test, macberta_y_test = torch.from_numpy(macberta_x_test).type(dtype), \
        torch.from_numpy(macberta_y_test).type(dtype)
mfcc_ft_x_test, mfcc_ft_y_test = torch.from_numpy(mfcc_ft_x_test).type(dtype), \
        torch.from_numpy(mfcc_ft_y_test).type(dtype)
face_x_test, face_y_test = torch.from_numpy(face_x_test).type(dtype), \
        torch.from_numpy(face_y_test).type(dtype)
iris_x_test, iris_y_test = torch.from_numpy(iris_x_test).type(dtype), \
        torch.from_numpy(iris_y_test).type(dtype)
physical_index_x_test, physical_index_y_test = torch.from_numpy(physical_index_x_test).type(dtype), \
        torch.from_numpy(physical_index_y_test).type(dtype)

y_train = pert_y_train
y_test = pert_y_test
y_train = y_train.unsqueeze(-1)
y_test = y_test.unsqueeze(-1)

    

In [5]:
from tsmoothie.smoother import *
import matplotlib.pyplot as plt
from numpy.fft import *
np.set_printoptions(threshold=np.inf)
smoother = ExponentialSmoother(window_len=20, alpha=0.3)
smoother = KalmanSmoother(component='level_trend', 
                          component_noise={'level':0.1, 'trend':0.1})

def filter_signal(signal, threshold=1e8):
    fourier = rfft(signal)
    frequencies = rfftfreq(signal.size, d=20e-3/signal.size)
    fourier[frequencies > threshold] = 0
    return irfft(fourier)

skf = StratifiedKFold(n_splits=10)
min_max_scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.StandardScaler()
x = np.array(face)
y = labels
y = np.array(y)
face_x_train_skf = []
face_x_test_skf = []
face_y_train_skf = []
face_y_test_skf = []
# pca = PCA(n_components=2)
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = scaler.fit_transform(x_train)
    # x_test = scaler.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    face_x_train_skf.append(x_train)
    face_x_test_skf.append(x_test)
    face_y_train_skf.append(y_train)
    face_y_test_skf.append(y_test)


print("face shape:", np.array(face_x_train_skf[0]).shape)
print("face shape:", np.array(face_x_test_skf[0]).shape)
x = np.array(iris)
y = np.array(y)
iris_x_train_skf = []
iris_x_test_skf = []
iris_y_train_skf = []
iris_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    # x_train = scaler.fit_transform(x_train)
    # x_test = scaler.fit_transform(x_test)
    iris_x_train_skf.append(x_train)
    iris_x_test_skf.append(x_test)
    iris_y_train_skf.append(y_train)
    iris_y_test_skf.append(y_test)


x = np.array(physical_index)
y = np.array(y)
physical_x_train_skf = []
physical_x_test_skf = []
physical_y_train_skf = []
physical_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)

    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    # x_train = scaler.fit_transform(x_train)
    # x_test = scaler.fit_transform(x_test)
    physical_x_train_skf.append(x_train)
    physical_x_test_skf.append(x_test)
    physical_y_train_skf.append(y_train)
    physical_y_test_skf.append(y_test)
print("physical index shape:", np.array(physical_x_train_skf[0]).shape)
print("physical index shape:", np.array(physical_x_test_skf[0]).shape)
x = np.array(mfcc_ft)
y = np.array(y)
mfcc_ft_x_train_skf = []
mfcc_ft_x_test_skf = []
mfcc_ft_y_train_skf = []
mfcc_ft_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    # x_train = scaler.fit_transform(x_train)
    # x_test = scaler.fit_transform(x_test)
    mfcc_ft_x_train_skf.append(x_train)
    mfcc_ft_x_test_skf.append(x_test)  
    mfcc_ft_y_train_skf.append(y_train)
    mfcc_ft_y_test_skf.append(y_test)
print("mfcc shape:", np.array(mfcc_ft_x_train_skf[0]).shape)
print("mfcc shape:", np.array(mfcc_ft_x_test_skf[0]).shape)
x = np.array(pert)
y = np.array(y)
pert_x_train_skf = []
pert_x_test_skf = []
pert_y_train_skf = []
pert_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    # x_train = scaler.fit_transform(x_train)
    # x_test = scaler.fit_transform(x_test)
    pert_x_train_skf.append(x_train)
    pert_x_test_skf.append(x_test)
    pert_y_train_skf.append(y_train)
    pert_y_test_skf.append(y_test)
print("pert shape:", np.array(pert_x_train_skf[0]).shape)
print("pert shape:", np.array(pert_x_test_skf[0]).shape)
x = np.array(roberta)
y = np.array(y)
roberta_x_train_skf = []
roberta_x_test_skf = []
roberta_y_train_skf = []
roberta_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    # x_train = scaler.fit_transform(x_train)
    # x_test = scaler.fit_transform(x_test)
    roberta_x_train_skf.append(x_train)
    roberta_x_test_skf.append(x_test)
    roberta_y_train_skf.append(y_train)
    roberta_y_test_skf.append(y_test)
print("roberta shape:", np.array(roberta_x_train_skf[0]).shape)
print("roberta shape:", np.array(roberta_x_test_skf[0]).shape)

x = np.array(macberta)
y = np.array(y)
macberta_x_train_skf = []
macberta_x_test_skf = []
macberta_y_train_skf = []
macberta_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)

    # for i in range(0, 375):
    #     x_train[i] = filter_signal(x_train[i], threshold=1e3)
    # for i in range(0, 41):
    #     x_test[i] = filter_signal(x_test[i], threshold=1e3)
    # smoother.smooth(x_train)
    # x_train = smoother.smooth_data
    # smoother.smooth(x_test)
    # x_test = smoother.smooth_data

    # x_train = pca.fit_transform(x_train)
    # x_test = pca.fit_transform(x_test)
    # x_train = preprocessing.scale(x_train)
    # x_test = preprocessing.scale(x_test)
    # x_train = scaler.fit_transform(x_train)
    # x_test = scaler.fit_transform(x_test)
    macberta_x_train_skf.append(x_train)
    macberta_x_test_skf.append(x_test)
    macberta_y_train_skf.append(y_train)
    macberta_y_test_skf.append(y_test)
print("macberta shape:", np.array(macberta_x_train_skf[0]).shape)
print("macberta shape:", np.array(macberta_x_test_skf[0]).shape)





face shape: (375, 1404)
face shape: (42, 1404)
physical index shape: (375, 4722)
physical index shape: (42, 4722)
mfcc shape: (375, 9828)
mfcc shape: (42, 9828)
pert shape: (375, 1024)
pert shape: (42, 1024)
roberta shape: (375, 1024)
roberta shape: (42, 1024)
macberta shape: (375, 1024)
macberta shape: (42, 1024)


In [6]:
face_x_train, face_x_test, face_y_train, face_y_test = \
        train_test_split(face, labels, random_state=0, train_size=0.7)
iris_x_train, iris_x_test, iris_y_train, iris_y_test = \
        train_test_split(iris, labels, random_state=0, train_size=0.7)
physical_index_x_train, physical_index_x_test, physical_index_y_train, physical_index_y_test = \
        train_test_split(physical_index, labels, random_state=0, train_size=0.7)
mfcc_ft_x_train, mfcc_ft_x_test, mfcc_ft_y_train, mfcc_ft_y_test = \
        train_test_split(mfcc_ft, labels, random_state=0, train_size=0.7)
text_x_train, text_x_test, text_y_train, text_y_test = \
        train_test_split(text, labels, random_state=0, train_size=0.7)

ros = RandomOverSampler(random_state=0)
sm = SMOTE(random_state=0)
face_x_train, face_y_train = ros.fit_resample(face_x_train, face_y_train)
iris_x_train, iris_y_train = ros.fit_resample(iris_x_train, iris_y_train)
physical_index_x_train, physical_index_y_train = ros.fit_resample(physical_index_x_train, \
        physical_index_y_train)
mfcc_ft_x_train, mfcc_ft_y_train = ros.fit_resample(mfcc_ft_x_train, mfcc_ft_y_train)
text_x_train, text_y_train = ros.fit_resample(text_x_train, text_y_train)

dtype = torch.FloatTensor

face_x_train, face_x_test, face_y_train, face_y_test = np.array(face_x_train), np.array(face_x_test),\
        np.array(face_y_train), np.array(face_y_test)
iris_x_train, iris_x_test, iris_y_train, iris_y_test = np.array(iris_x_train), np.array(iris_x_test),\
        np.array(iris_y_train), np.array(iris_y_test)
physical_index_x_train, physical_index_x_test, physical_index_y_train, physical_index_y_test = \
        np.array(physical_index_x_train), np.array(physical_index_x_test), \
            np.array(physical_index_y_train), np.array(physical_index_y_test)
mfcc_ft_x_train, mfcc_ft_x_test, mfcc_ft_y_train, mfcc_ft_y_test = np.array(mfcc_ft_x_train), \
        np.array(mfcc_ft_x_test), np.array(mfcc_ft_y_train), np.array(mfcc_ft_y_test)
text_x_train, text_x_test, text_y_train, text_y_test = np.array(text_x_train), np.array(text_x_test),\
        np.array(text_y_train), np.array(text_y_test)

text_x_train, text_y_train = torch.from_numpy(text_x_train).type(dtype), \
        torch.from_numpy(text_y_train).type(dtype)
mfcc_ft_x_train, mfcc_ft_y_train = torch.from_numpy(mfcc_ft_x_train).type(dtype), \
        torch.from_numpy(mfcc_ft_y_train).type(dtype)
face_x_train, face_y_train = torch.from_numpy(face_x_train).type(dtype), \
        torch.from_numpy(face_y_train).type(dtype)
iris_x_train, iris_y_train = torch.from_numpy(iris_x_train).type(dtype), \
        torch.from_numpy(iris_y_train).type(dtype)
physical_index_x_train, physical_index_y_train = torch.from_numpy(physical_index_x_train).type(dtype), \
        torch.from_numpy(physical_index_y_train).type(dtype)

text_x_test, text_y_test = torch.from_numpy(text_x_test).type(dtype), \
        torch.from_numpy(text_y_test).type(dtype)
mfcc_ft_x_test, mfcc_ft_y_test = torch.from_numpy(mfcc_ft_x_test).type(dtype), \
        torch.from_numpy(mfcc_ft_y_test).type(dtype)
face_x_test, face_y_test = torch.from_numpy(face_x_test).type(dtype), \
        torch.from_numpy(face_y_test).type(dtype)
iris_x_test, iris_y_test = torch.from_numpy(iris_x_test).type(dtype), \
        torch.from_numpy(iris_y_test).type(dtype)
physical_index_x_test, physical_index_y_test = torch.from_numpy(physical_index_x_test).type(dtype), \
        torch.from_numpy(physical_index_y_test).type(dtype)

y_train = text_y_train
y_test = text_y_test
y_train = y_train.unsqueeze(-1)
y_test = y_test.unsqueeze(-1)

    

NameError: name 'text' is not defined

In [7]:
skf = StratifiedKFold(n_splits=10)
min_max_scaler = preprocessing.MinMaxScaler()
x = np.array(face)
y = labels
y = np.array(y)
face_x_train_skf = []
face_x_test_skf = []
face_y_train_skf = []
face_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    face_x_train_skf.append(x_train)
    face_x_test_skf.append(x_test)
    face_y_train_skf.append(y_train)
    face_y_test_skf.append(y_test)

x = np.array(iris)
y = np.array(y)
iris_x_train_skf = []
iris_x_test_skf = []
iris_y_train_skf = []
iris_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    iris_x_train_skf.append(x_train)
    iris_x_test_skf.append(x_test)
    iris_y_train_skf.append(y_train)
    iris_y_test_skf.append(y_test)

x = np.array(physical_index)
y = np.array(y)
physical_x_train_skf = []
physical_x_test_skf = []
physical_y_train_skf = []
physical_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    physical_x_train_skf.append(x_train)
    physical_x_test_skf.append(x_test)
    physical_y_train_skf.append(y_train)
    physical_y_test_skf.append(y_test)

x = np.array(mfcc_ft)
y = np.array(y)
mfcc_ft_x_train_skf = []
mfcc_ft_x_test_skf = []
mfcc_ft_y_train_skf = []
mfcc_ft_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    mfcc_ft_x_train_skf.append(x_train)
    mfcc_ft_x_test_skf.append(x_test)  
    mfcc_ft_y_train_skf.append(y_train)
    mfcc_ft_y_test_skf.append(y_test)

x = np.array(text)
y = np.array(y)
text_x_train_skf = []
text_x_test_skf = []
text_y_train_skf = []
text_y_test_skf = []
for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.fit_transform(x_test)
    text_x_train_skf.append(x_train)
    text_x_test_skf.append(x_test)
    text_y_train_skf.append(y_train)
    text_y_test_skf.append(y_test)

# face_train, face_test = np.array(face_train), np.array(face_test)
# iris_train, iris_test = np.array(iris_train), np.array(iris_test)
# physical_train, physical_test = np.array(physical_train), np.array(physical_test)
# mfcc_ft_train, mfcc_ft_test = np.array(mfcc_ft_train), np.array(mfcc_ft_test)
# text_train, text_test = np.array(text_train), np.array(text_test)

# y_train = text_y_train
# y_test = text_y_test
# y_train = y_train.unsqueeze(-1)
# y_test = y_test.unsqueeze(-1)

NameError: name 'text' is not defined

In [6]:
model = TFN(input_dims=(9828, 1404, 1024, 4, 4722, 1024, 1024), hidden_dims=(4, 4, 4, 4, 4, 4, 4),\
         dropouts=(0.2, 0.1, 0, 0.3, 0.3, 0.1, 0.1, 0.5), post_fusion_dim=32)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [8]:
total_acc = []
for j in range(0, 1):
        ACC = []
        for i in range(0, 10):
                print("The {round} group of {time}".format(round=i, time = j))
                sm = SMOTE(random_state=0)
                ros = RandomOverSampler(random_state=0)
                face_x_train, face_y_train = ros.fit_resample(face_x_train_skf[i], face_y_train_skf[i])
                iris_x_train, iris_y_train = ros.fit_resample(iris_x_train_skf[i], iris_y_train_skf[i])
                physical_x_train, physical_y_train = ros.fit_resample(physical_x_train_skf[i], \
                        physical_y_train_skf[i])
                mfcc_ft_x_train, mfcc_ft_y_train = ros.fit_resample(mfcc_ft_x_train_skf[i], mfcc_ft_y_train_skf[i])
                pert_x_train, pert_y_train = ros.fit_resample(pert_x_train_skf[i], pert_y_train_skf[i])
                macberta_x_train, macberta_y_train = ros.fit_resample(macberta_x_train_skf[i], macberta_y_train_skf[i])
                roberta_x_train, roberta_y_train = ros.fit_resample(roberta_x_train_skf[i], roberta_y_train_skf[i])

                dtype = torch.FloatTensor

                face_x_train, face_x_test, face_y_train, face_y_test = np.array(face_x_train), np.array(face_x_test_skf[i]),\
                        np.array(face_y_train), np.array(face_y_test_skf[i])
                iris_x_train, iris_x_test, iris_y_train, iris_y_test = np.array(iris_x_train), np.array(iris_x_test_skf[i]),\
                        np.array(iris_y_train), np.array(iris_y_test_skf[i])
                physical_index_x_train, physical_index_x_test, physical_index_y_train, physical_index_y_test = \
                        np.array(physical_x_train), np.array(physical_x_test_skf[i]), \
                        np.array(physical_y_train), np.array(physical_y_test_skf[i])
                mfcc_ft_x_train, mfcc_ft_x_test, mfcc_ft_y_train, mfcc_ft_y_test = np.array(mfcc_ft_x_train), \
                        np.array(mfcc_ft_x_test_skf[i]), np.array(mfcc_ft_y_train), np.array(mfcc_ft_y_test_skf[i])
                pert_x_train, pert_x_test, pert_y_train, pert_y_test = np.array(pert_x_train), np.array(pert_x_test_skf[i]),\
                        np.array(pert_y_train), np.array(pert_y_test_skf[i])
                macberta_x_train, macberta_x_test, macberta_y_train, macberta_y_test = np.array(macberta_x_train), np.array(macberta_x_test_skf[i]),\
                        np.array(macberta_y_train), np.array(macberta_y_test_skf[i])
                roberta_x_train, roberta_x_test, roberta_y_train, roberta_y_test = np.array(roberta_x_train), np.array(roberta_x_test_skf[i]),\
                        np.array(roberta_y_train), np.array(roberta_y_test_skf[i])

                pert_x_train, pert_y_train = torch.from_numpy(pert_x_train).type(dtype), \
                        torch.from_numpy(pert_y_train).type(dtype)
                macberta_x_train, macberta_y_train = torch.from_numpy(macberta_x_train).type(dtype), \
                        torch.from_numpy(macberta_y_train).type(dtype)
                roberta_x_train, roberta_y_train = torch.from_numpy(roberta_x_train).type(dtype), \
                        torch.from_numpy(roberta_y_train).type(dtype)
                mfcc_ft_x_train, mfcc_ft_y_train = torch.from_numpy(mfcc_ft_x_train).type(dtype), \
                        torch.from_numpy(mfcc_ft_y_train).type(dtype)
                face_x_train, face_y_train = torch.from_numpy(face_x_train).type(dtype), \
                        torch.from_numpy(face_y_train).type(dtype)
                iris_x_train, iris_y_train = torch.from_numpy(iris_x_train).type(dtype), \
                        torch.from_numpy(iris_y_train).type(dtype)
                physical_index_x_train, physical_index_y_train = torch.from_numpy(physical_index_x_train).type(dtype), \
                        torch.from_numpy(physical_index_y_train).type(dtype)

                pert_x_test, pert_y_test = torch.from_numpy(pert_x_test).type(dtype), \
                        torch.from_numpy(pert_y_test).type(dtype)
                macberta_x_test, macberta_y_test = torch.from_numpy(macberta_x_test).type(dtype), \
                        torch.from_numpy(macberta_y_test).type(dtype)
                roberta_x_test, roberta_y_test = torch.from_numpy(roberta_x_test).type(dtype), \
                        torch.from_numpy(roberta_y_test).type(dtype)
                mfcc_ft_x_test, mfcc_ft_y_test = torch.from_numpy(mfcc_ft_x_test).type(dtype), \
                        torch.from_numpy(mfcc_ft_y_test).type(dtype)
                face_x_test, face_y_test = torch.from_numpy(face_x_test).type(dtype), \
                        torch.from_numpy(face_y_test).type(dtype)
                iris_x_test, iris_y_test = torch.from_numpy(iris_x_test).type(dtype), \
                        torch.from_numpy(iris_y_test).type(dtype)
                physical_index_x_test, physical_index_y_test = torch.from_numpy(physical_index_x_test).type(dtype), \
                        torch.from_numpy(physical_index_y_test).type(dtype)

                y_train = pert_y_train
                y_test = pert_y_test
                y_train = y_train.unsqueeze(-1)
                y_test = y_test.unsqueeze(-1)

                model = TFN(input_dims=(9828, 1404, 1024, 2736, 4722, 1024, 1024), hidden_dims=(4,4,4,4,4, 4, 4),\
         dropouts=(0.1, 0.1, 0.1, 0.5, 0.5, 0.1, 0.1, 0.5), post_fusion_dim=1)
                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
                model.to(device)

                loss_func = torch.nn.L1Loss()
                optimizer = torch.optim.Adam(model.parameters())

                epochs = 10000
                temp_acc = 0  
                for t in range(epochs+1):
                        model.train(True)
                        out = model(mfcc_ft_x_train.to(device), face_x_train.to(device), pert_x_train.to(device), \
                                iris_x_train.to(device), physical_index_x_train.to(device), macberta_x_train.to(device), \
                                        roberta_x_train.to(device))
                        loss = loss_func(out, y_train.to(device))
                        # print("train loss:%f" %loss.detach().numpy())
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                        if t % 10 == 0 and t > 0:
                                model.train(False)
                                pred_y = out.detach().cpu().numpy()
                                for item in range(pred_y.shape[0]):
                                        if pred_y[item] > 0.5:
                                                pred_y[item] = 1
                                        else:
                                                pred_y[item] = 0
                                target_y = y_train.data.numpy()
                                print("epoch %d, train acc %.4f" % (t, accuracy_score(pred_y, target_y)))

                
                                out = model(mfcc_ft_x_test.to(device), face_x_test.to(device), pert_x_test.to(device), \
                                        iris_x_test.to(device), physical_index_x_test.to(device), macberta_x_test.to(device), \
                                                roberta_x_test.to(device))
                                loss = loss_func(out,  y_test.to(device))
                                print("test loss:%f" %loss.detach().cpu().numpy())
                
                                pred_y = out.detach().cpu().numpy()
                                for item in range(pred_y.shape[0]):
                                        if pred_y[item] > 0.5:
                                                pred_y[item] = 1
                                        else:
                                                pred_y[item] = 0
                                target_y = y_test.data.numpy()
                                test_score = accuracy_score(target_y, pred_y)
                                if test_score > temp_acc:
                                        acc =  test_score
                                        temp_acc = test_score
                                        print(acc)
                                print("test acc %.6f" % test_score)
                                # test_f1 = f1_score(target_y, pred_y, average='weighted')
                #     print("test f1-score %f" %test_f1 )
                ACC.append(acc)
        # print("LMF Max score:", np.max(ACC))
        # print("LMF Min score:", np.min(ACC))
        print("LMF Avg score:", np.mean(ACC))
        total_acc.append(np.mean(ACC))
print("Average acc:{avg}, Best acc:{best}".format(avg=np.mean(total_acc), best=np.max(total_acc)))
        

The 0 group of 0
epoch 10, train acc 0.5000
test loss:0.506488
0.21428571428571427
test acc 0.214286
epoch 20, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 30, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 40, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 50, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 60, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 70, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 80, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 90, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 100, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 110, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 120, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 130, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 140, train acc 0.5000
test loss:0.506488
test acc 0.214286
epoch 150, train acc 0.5000
test loss:0.506488
test acc 0.2142

KeyboardInterrupt: 